# CIS Data to Google Earth (Real Time)

In [ ]:
# TODO: Functionality with meters
# TODO: Include shorted status in exception report
# TODO: Delete (250V: Range from Comments)

In [ ]:
# Standard library imports for filesystem and execution time
import os
import shutil
import time
import datetime

# Standard library import for networking
import socket

# Standard library imports for mathematical functions
import math
import scipy.stats as stats
from math import atan, atan2, cos, radians, sin, sqrt, tan

# Third-party imports for data handling
import pandas as pd
import numpy as np
from natsort import natsorted
from collections import OrderedDict

# Third-party imports for KML (Keyhole Markup Language) and Excel (`xlsx`) files handling
import simplekml
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, Border, Side

# Third-party import for filesystem path names pattern matching
import glob

# Settings for pandas
pd.options.mode.chained_assignment = None

# Measuring execution time
start_time = time.time()

## Directories

In [ ]:
# Folders
ROOT_DIR = os.path.abspath('../')
DESKTOP_DIR = os.path.join(os.path.expanduser("~"), 'Desktop')
ORIGINAL_DATA_DIR = os.path.join(ROOT_DIR, 'Original Data')
DATA_DIR = os.path.join(ROOT_DIR, 'Data')
OUTPUT_DIR = os.path.join(DESKTOP_DIR, 'Output')

In [ ]:
# Files
LOG_DIR = os.path.join(OUTPUT_DIR, 'Log.txt')
DUPLICATES_DIR = os.path.join(OUTPUT_DIR, 'Duplicate GPS Pairs.csv')

## Variables

In [ ]:
# CLIENT
CLIENT = 'Kinder Morgan'

# VARIABLE TO ENABLE 3D PLOTTING (True or False)
PLOT_3D = True

# REVERSE DATA ROWS (True or False)
REVERSE = False

# DATA VISIBILITY WHEN OPENING KMZ FILE (True or False)
DATA_VISIBILITY = True

# DISTANCE (FT) CUTOFFS BETWEEN GPS POINTS TO BE USED FOR STATISTICS
LOWER_CUTOFF = 1
UPPER_CUTOFF = 3

In [ ]:
# ICONS
ICON_ON = 'https://img.icons8.com/ios-filled/50/FC9CFF/filled-circle.png'
ICON_OFF = 'https://img.icons8.com/ios-filled/50/00FF00/filled-circle.png'
ICON_NATIVE = 'https://img.icons8.com/ios-filled/50/175082/filled-circle.png'
ICON_1200 = 'https://img.icons8.com/ios-filled/50/7950F2/filled-circle.png'
ICON_850 = 'https://img.icons8.com/ios-filled/50/F25081/filled-circle.png'
ICON_COMMENTS = 'https://img.icons8.com/ultraviolet/80/000000/comments.png'
ICON_ACVG = 'https://img.icons8.com/material-rounded/50/6BFF00/sine.png'
ICON_PCM = 'https://img.icons8.com/ios-filled/50/FFFFFF/lightning-bolt--v1.png'
ICON_PCM_PERCENT = 'https://img.icons8.com/fluency-systems-regular/50/FFFFFF/percentage-circle.png'

# SCALE FACTOR (Elevation of points for -0.85V to be 100m, Default = 117.64705882352942) 
SCALE_FACTOR = 117.64705882352942

# PCM SCALE FACTOR (Elevation of points for max value of 3 to be 100m, Default = 100 / 3)
SCALE_PCM = 100 / 3

# PCM % SCALE FACTOR (Elevation of points for max value of 100% to be 100m, Default = 50)
SCALE_PCM_PERCENT = 50

# ICON SCALE (Default = 0.3)
ICON_SCALE = 0.2

## Vincenty's Formula

In [ ]:
class VincentyInverse:
    def __init__(self, coord1, coord2, maxiter=200, tol=10 ** -12):
        # CONSTANTS
        a = 6378137.0  # radius at equator in meters (WGS-84)
        f = 1 / 298.257223563  # flattening of the ellipsoid (WGS-84)
        b = (1 - f) * a
        phi_1, lon_1, = coord1  # (lat=L_?,lon=phi_?)
        phi_2, lon_2, = coord2
        u_1 = atan((1 - f) * tan(radians(phi_1)))
        u_2 = atan((1 - f) * tan(radians(phi_2)))
        L = radians(lon_2 - lon_1)
        lamda = L  # set initial value of lambda to L
        sin_u1 = sin(u_1)
        cos_u1 = cos(u_1)
        sin_u2 = sin(u_2)
        cos_u2 = cos(u_2)

        # ITERATIONS
        self.iters = 0
        for i in range(0, maxiter):
            self.iters += 1
            cos_lambda = cos(lamda)
            sin_lambda = sin(lamda)
            sin_sigma = sqrt((cos_u2 * sin(lamda)) ** 2 +
                             (cos_u1 * sin_u2 - sin_u1 * cos_u2 * cos_lambda) ** 2)
            cos_sigma = sin_u1 * sin_u2 + cos_u1 * cos_u2 * cos_lambda
            sigma = atan2(sin_sigma, cos_sigma)
            sin_alpha = (cos_u1 * cos_u2 * sin_lambda) / sin_sigma
            cos_sq_alpha = 1 - sin_alpha ** 2
            cos2_sigma_m = cos_sigma - ((2 * sin_u1 * sin_u2) / cos_sq_alpha)
            C = (f / 16) * cos_sq_alpha * (4 + f * (4 - 3 * cos_sq_alpha))
            Lambda_prev = lamda
            lamda = L + (1 - C) * f * sin_alpha * (sigma + C * sin_sigma *
                                                   (cos2_sigma_m + C * cos_sigma *
                                                    (-1 + 2 * cos2_sigma_m ** 2)))

            # Successful convergence
            diff = abs(Lambda_prev - lamda)
            if diff <= tol:
                break

        u_sq = cos_sq_alpha * ((a ** 2 - b ** 2) / b ** 2)
        A = 1 + (u_sq / 16384) * (4096 + u_sq * (-768 + u_sq * (320 - 175 * u_sq)))
        B = (u_sq / 1024) * (256 + u_sq * (-128 + u_sq * (74 - 47 * u_sq)))
        delta_sig = B * sin_sigma * (cos2_sigma_m + 0.25 * B *
                                     (cos_sigma * (-1 + 2 * cos2_sigma_m ** 2) -
                                      (1 / 6) * B * cos2_sigma_m * (-3 + 4 * sin_sigma ** 2) *
                                      (-3 + 4 * cos2_sigma_m ** 2)))

        self.m = b * A * (sigma - delta_sig)  # Meters
        self.miles = self.m * 0.000621371  # Miles
        self.ft = self.miles * 5280  # Feet

# Data Cleaning

## Directory Management

In [ ]:
# Replaces data folder with an empty one
if os.path.exists(DATA_DIR):
    shutil.rmtree(DATA_DIR)

os.makedirs(DATA_DIR)

In [ ]:
# Replaces output folder with an empty one
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR)

## Log File

In [ ]:
# Removes existing log file, if it exists
if os.path.exists(LOG_DIR):
    os.remove(LOG_DIR)

# Creates a log file to track information of interest
with open(LOG_DIR, 'a') as f:
    f.write("Rows dropped with missing GPS points: ")

## Clean Raw Files

In [ ]:
# List of files contained in the original data folder
raw_data_list = os.listdir(ORIGINAL_DATA_DIR)

# List for files excluding ones starting with '.'
filtered_data_list = [file for file in raw_data_list if not file.startswith('.')]

# Removes '.DAT' string and changes '.CSV' to lower case in any files that have it
for idx, name in enumerate(filtered_data_list):
    original_name = os.path.join(ORIGINAL_DATA_DIR, name)
    new_name = os.path.join(ORIGINAL_DATA_DIR, name.replace('.DAT', '').replace('.CSV', '.csv'))
    os.rename(original_name, new_name)
    filtered_data_list[idx] = name.replace('.DAT', '').replace('.CSV', '.csv')

# Sort
filtered_data_list = natsorted(filtered_data_list)

## Dataframe

In [ ]:
total_miles_list = []  # List for total miles per '.csv' file
total_rows_dropped = []  # List for total rows dropped that don't have GPS coordinates
i = 0  # Loop counter for station numbers
j = 0  # List counter for '.csv' files

# Goes through each '.csv' in the folder
while j < len(filtered_data_list):
    # Create dataframe from a csv file
    df_cis = pd.read_csv(os.path.join(ORIGINAL_DATA_DIR, filtered_data_list[j]))

    # Rename columns
    rename_dict = {
        'records': 'Station',
        'milepost': 'Station',
        'ps on': 'On Potential',
        'structureps': 'On Potential',
        'ps off': 'Off Potential',
        'structureirf': 'Off Potential',
        'comment': 'Comments',
        'comments': 'Comments',
        'locationdescription': 'Comments',
        'latitude': 'Latitude',
        'longitude': 'Longitude'
    }

    # Replace original names to correct format
    df_cis.columns = [rename_dict.get(x.lower(), x) for x in df_cis.columns]

    # Columns for dataframe
    cols = ['Station', 'On Potential', 'Off Potential', 'Native',
            'Comments', 'Latitude', 'Longitude',
            'ACVG Indication (dB_V)', 'ACVG Notes', 'PCM Data (Amps)', 'PCM % Change']

    # Create dataframe from columns that exist
    df_cis = df_cis[df_cis.columns.intersection(cols)]

    # Reversing rows for flipped SN
    if REVERSE:
        # Reversing rows
        df_cis.iloc[:, :] = df_cis.iloc[:, :].values[::-1]

        # Reordering
        df_cis['Station'] = abs(max(df_cis['Station']) - df_cis['Station'])

    # Desired columns
    cols = ['On Potential', 'Off Potential', 'Native']

    for col in cols:
        if col in df_cis.columns:
            # Delete rows that CIS was skipped and reset the index
            df_cis = df_cis.loc[df_cis[col] != 'SKIP'].reset_index(drop=True)

            # Convert object columns to numbers
            df_cis[col] = pd.to_numeric(df_cis[col], errors='coerce')

            # Replace empty values with 0
            df_cis[col] = df_cis[col].replace(np.nan, 0)

            # Convert positive values to negative
            df_cis[col] = df_cis[col].abs() * (-1)

            # Replace exact values
            df_cis[col] = df_cis[col].replace(-0.85, -0.850001)
            df_cis[col] = df_cis[col].replace(-1.2, -1.20001)

        # Divide by 1000 if needed
        if col in df_cis.columns and abs(df_cis[col].mean()) > 100:
            df_cis[col] /= 1000

    # Trim white space from comments
    df_cis['Comments'] = df_cis['Comments'].str.strip()

    # Create 'Distance (ft)' column
    df_cis['Distance (ft)'] = 0.00

    if 'ACVG Indication (dB_V)' in df_cis.columns:
        # Convert object columns to numbers
        df_cis['ACVG Indication (dB_V)'] = pd.to_numeric(df_cis['ACVG Indication (dB_V)'],
                                                         errors='coerce')

        # Replace empty values with 0
        df_cis['ACVG Indication (dB_V)'] = df_cis['ACVG Indication (dB_V)'].replace(np.nan, 0)

        # Trim white space from comments
        df_cis['ACVG Notes'] = df_cis['ACVG Notes'].str.strip()

    if 'PCM Data (Amps)' in df_cis.columns:
        # Convert object columns to numbers
        df_cis['PCM Data (Amps)'] = pd.to_numeric(df_cis['PCM Data (Amps)'], errors='coerce')

        # Replace empty values with 0
        df_cis['PCM Data (Amps)'] = df_cis['PCM Data (Amps)'].replace(np.nan, 0)

    if 'PCM % Change' in df_cis.columns:
        # Convert object columns to numbers
        df_cis['PCM % Change'] = pd.to_numeric(df_cis['PCM % Change'], errors='coerce')

        # Replace empty values with 0
        df_cis['PCM % Change'] = df_cis['PCM % Change'].replace(np.nan, 0)

        # Absolute value
        df_cis['PCM % Change'] = abs(df_cis['PCM % Change'])

    # Initial rows 
    last_index2 = df_cis.last_valid_index()

    # Drop rows that don't have GPS coordinates
    df_cis['Latitude'] = df_cis['Latitude'].replace('', np.nan)
    df_cis.dropna(subset=['Latitude'], inplace=True)
    df_cis.reset_index(drop=True, inplace=True)
    last_index = df_cis.last_valid_index()
    total_rows_dropped.append(last_index2 - last_index)

    # Record number of rows dropped for each file
    with open(LOG_DIR, 'a') as f:
        f.write(f"{total_rows_dropped[j]}\n\n")

    #----------------------------------------------------------------------------------------------
    # STATION NUMBERS

    # Extract starting station number from '.csv' title
    starting_stationing = float(filtered_data_list[j].split('SN')[1].split()[0].replace('+', ''))

    # Add 'Stationing (ft)' column to hold calculated stationing values
    df_cis['Stationing (ft)'] = ''

    # Infers actual station number (XX+XX)
    while i <= last_index:
        current_station = df_cis.at[i, 'Station']
        stationing_ft = str(starting_stationing + current_station).split('.')[0]
        string_length = len(stationing_ft)

        if -100 < (starting_stationing + current_station) < 0:
            station_value = stationing_ft.replace('-', '')
            df_cis.at[i, 'Stationing (ft)'] = (
                f" -0{station_value[:string_length - 2]}+{station_value[string_length - 2:]}")

        elif 0 <= (starting_stationing + current_station) < 100:
            df_cis.at[i, 'Stationing (ft)'] = (
                f" 0{stationing_ft[:string_length - 2]}+{stationing_ft[string_length - 2:]}")

        else:
            df_cis.at[i, 'Stationing (ft)'] = (
                f"{stationing_ft[:string_length - 2]}+{stationing_ft[string_length - 2:]}")

        # Last row exception
        if i != last_index:
            # Calculates distance between GPS coordinates and stationing (ft)
            try:
                df_cis.at[i + 1, 'Distance (ft)'] = VincentyInverse(
                    [df_cis.at[i, 'Latitude'], df_cis.at[i, 'Longitude']],
                    [df_cis.at[i + 1, 'Latitude'], df_cis.at[i + 1, 'Longitude']]).ft

            except ZeroDivisionError:
                df_cis.at[i + 1, 'Distance (ft)'] = 0

        # Counters
        i += 1

    #----------------------------------------------------------------------------------------------
    # STATISTICS
    df_cis_stats = df_cis.copy()
    df_cis_stats = df_cis_stats[(LOWER_CUTOFF < df_cis_stats['Station'].diff().abs()) &
                                (df_cis_stats['Station'].diff().abs() <= UPPER_CUTOFF)]

    # Mean
    mean = df_cis_stats['Distance (ft)'][1:].mean()

    # Mode
    mode = df_cis_stats['Distance (ft)'].round(1).mode()

    # Standard deviation
    std = df_cis_stats['Distance (ft)'][1:].std()

    # Z-score (Identify outliers)
    z_scores = np.abs(stats.zscore(df_cis_stats['Distance (ft)']))
    df_cis_stats['z_scores'] = z_scores
    df_outliers = df_cis_stats[df_cis_stats['z_scores'] > 2.576][['Stationing (ft)',
                                                                  'Distance (ft)']]

    # Total count of data within the distance cutoff
    total_count = df_cis_stats['Distance (ft)'].round(1).count()

    # Count of data that fall outside the cutoff
    cutoff_count = df_cis_stats[(LOWER_CUTOFF < df_cis_stats['Distance (ft)']) &
                                (df_cis_stats['Distance (ft)'] <= UPPER_CUTOFF)].count().iat[0]

    # Records total miles in a list
    total_miles_list.append(max(df_cis['Station']) / 5280)

    # Export to '.csv'
    df_cis.to_csv(os.path.join(DATA_DIR, filtered_data_list[j]), index=False)
    df_outliers.to_csv(os.path.join(OUTPUT_DIR, 'GPS Outliers.csv'), index=False)

    # Counters
    i = 0
    j += 1

## Create Google Earth Folders

In [ ]:
cis_kmz = []  # List for '.kmz' files
type_folders = []  # List for folders
total_folders = 9
current_mile = 1
k = 0  # List counter for type folders
j = 0  # List counter for '.csv' files
i = 0  # List counter for '.kmz' files based on miles

# Create CIS survey '.kmz' files per mile #
while j < len(total_miles_list):
    miles_remaining = round(total_miles_list[j], 2)

    # Create type folders
    while miles_remaining > 0:
        # Separate folders by mile
        cis_kmz.append(f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile})")
        cis_kmz[i] = simplekml.Kml()

        # Create list placeholders for folders
        for l in range(total_folders):
            type_folders.append('')

        type_folders[k + 0] = cis_kmz[i].newfolder(name='On')
        type_folders[k + 1] = cis_kmz[i].newfolder(name='Off')

        if 'Native' in df_cis.columns:
            type_folders[k + 2] = cis_kmz[i].newfolder(name='Native')

        type_folders[k + 3] = cis_kmz[i].newfolder(name='Comments')
        type_folders[k + 4] = cis_kmz[i].newfolder(name='-1.2 V')
        type_folders[k + 5] = cis_kmz[i].newfolder(name='-0.85 V')

        if 'ACVG Indication (dB_V)' in df_cis.columns:
            type_folders[k + 6] = cis_kmz[i].newfolder(name='ACVG Indication')

        if 'PCM Data (Amps)' in df_cis.columns:
            type_folders[k + 7] = cis_kmz[i].newfolder(name='PCM (Amps)')

        if 'PCM % Change' in df_cis.columns:
            type_folders[k + 8] = cis_kmz[i].newfolder(name='PCM (%)')

        # Save to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[i].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        i += 1
        k += total_folders

    # Counters
    j += 1
    current_mile = 1

# CIS Data

## On

In [ ]:
feet_counter = 5280
style = simplekml.Style()
i = 0  # Loop counter for rows
j = 0  # Loop counter for '.csv' files
k = 0  # Loop counter for type folders (0 = 'On')
kmz_file = 0

# Goes through all '.csv' files
while j < len(filtered_data_list) and 'On Potential' in df_cis.columns and PLOT_3D:
    df_cis_on = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[j]))
    df_cis_on = df_cis_on[df_cis_on['On Potential'] != 0]
    df_cis_on = df_cis_on[['Station', 'Stationing (ft)', 'Comments', 'Longitude',
                           'Latitude', 'On Potential']].reset_index(drop=True)
    df_cis_on['On Potential'] = df_cis_on['On Potential'] * (-1)
    last_index = df_cis_on.last_valid_index()
    on_measurements = last_index + 1
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_on.at[i, 'Station'] < feet_counter:
            # Break loop if there are no 'On' potentials
            if df_cis_on.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_on.at[i, 'On Potential'] * (-1),
                                           visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"Potential (On): -{df_cis_on.at[i, 'On Potential']} V\n"
                f"Longitude: {df_cis_on.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_on.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_on.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_on.at[i, 'Longitude'], df_cis_on.at[i, 'Latitude'],
                           df_cis_on.at[i, 'On Potential'] * SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_ON
            pnt.style.iconstyle.scale = ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0

            # Counters
            i += 1

        # Save to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

In [ ]:
# Record duplicate GPS coordinates in a '.csv' file
if PLOT_3D:
    df_duplicates = df_cis_on[df_cis_on.duplicated(subset=['Longitude', 'Latitude'], keep=False)]
    df_duplicates.to_csv(DUPLICATES_DIR, index=False)

## Off

In [ ]:
k = 1  # Loop counter for type folders (1 = 'Off')
off_measurements = 0

# Goes through all '.csv' files
while j < len(filtered_data_list) and 'Off Potential' in df_cis.columns and PLOT_3D:
    df_cis_off = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[j]))
    df_cis_off = df_cis_off[df_cis_off['Off Potential'] != 0]
    df_cis_off = df_cis_off[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                             'Off Potential']].reset_index(drop=True)
    df_cis_off['Off Potential'] = df_cis_off['Off Potential'] * (-1)
    last_index = df_cis_off.last_valid_index()
    off_measurements = last_index + 1
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_off.at[i, 'Station'] < feet_counter:
            # Break loop if there are no 'Off' potentials
            if df_cis_off.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_off.at[i, 'Off Potential'] * (-1),
                                           visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"Potential (Off): -{df_cis_off.at[i, 'Off Potential']} V\n"
                f"Longitude: {df_cis_off.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_off.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_off.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_off.at[i, 'Longitude'], df_cis_off.at[i, 'Latitude'],
                           df_cis_off.at[i, 'Off Potential'] * SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_OFF
            pnt.style.iconstyle.scale = ICON_SCALE
            pnt.style.linestyle.width = 0.01
            pnt.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, round(255 * 0.15))
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 1

            # Counters
            i += 1

        # Save to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## Native

In [ ]:
k = 2  # Loop counter for type folders (2 = 'Native')

# Goes through all '.csv' files
while j < len(filtered_data_list) and 'Native' in df_cis.columns and PLOT_3D:
    df_cis_native = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[j]))
    df_cis_native = df_cis_native[df_cis_native['Native'] != 0]
    df_cis_native = df_cis_native[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                                   'Native']].reset_index(drop=True)
    df_cis_native['Native'] = df_cis_native['Native'] * (-1)
    last_index = df_cis_native.last_valid_index()
    off_measurements = last_index + 1
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_native.at[i, 'Station'] < feet_counter:
            # Break loop if there are no 'Native' potentials
            if df_cis_native.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_native.at[i, 'Native'] * (-1),
                                           visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"Potential (Native): -{df_cis_native.at[i, 'Native']} V\n"
                f"Longitude: {df_cis_native.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_native.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_native.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_native.at[i, 'Longitude'], df_cis_native.at[i, 'Latitude'],
                           df_cis_native.at[i, 'Native'] * SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_NATIVE
            pnt.style.iconstyle.scale = ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0

            # Counters
            i += 1

        # Save to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## Comments

In [ ]:
k = 3  # Loop counter for type folders (3 = 'Comments')

# Goes through all '.csv' files
while j < len(filtered_data_list) and 'Comments' in df_cis.columns and PLOT_3D:
    df_cis_comments = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[j]))
    df_cis_comments.dropna(subset=['Comments'], inplace=True)
    df_cis_comments.reset_index(drop=True, inplace=True)
    last_index = df_cis_comments.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_comments.at[i, 'Station'] < feet_counter:
            # Break loop if there are no comments
            if df_cis_comments.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_comments.at[i, 'Comments'],
                                           visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = ''

            if 'On Potential' in df_cis.columns:
                potential_on = df_cis_comments.at[i, 'On Potential'].__str__() + ' V'
                pnt.style.balloonstyle.text += f"Potential (On): {potential_on}\n"

            if 'Off Potential' in df_cis.columns:
                potential_off = df_cis_comments.at[i, 'Off Potential'].__str__() + ' V'
                pnt.style.balloonstyle.text += f"Potential (Off): {potential_off}\n"

            if 'Native' in df_cis.columns:
                potential_native = df_cis_comments.at[i, 'Native'].__str__() + ' V'
                pnt.style.balloonstyle.text += f"Potential (Native): {potential_native}\n"

            pnt.style.balloonstyle.text += (
                f"Longitude: {df_cis_comments.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_comments.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_comments.at[i, 'Stationing (ft)']}\n"
                f"Comment: {df_cis_comments.at[i, 'Comments']}")

            pnt.coords = [(df_cis_comments.at[i, 'Longitude'],
                           df_cis_comments.at[i, 'Latitude'], 0)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_COMMENTS
            pnt.style.iconstyle.scale = ICON_SCALE * 1.5
            pnt.style.labelstyle.scale = 0.5

            # Counters
            i += 1

        # Export to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## -1.2 V

In [ ]:
k = 4  # Loop counter for type folders (4 = '-1.2 V')

# Goes through all '.csv' files
while j < len(filtered_data_list) and PLOT_3D:
    df_cis_1200 = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[j]))
    last_index = df_cis_1200.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_1200.at[i, 'Station'] < feet_counter:
            pnt = type_folders[k].newpoint(name='-1.200', visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = 'Potential: -1.2 V'
            pnt.coords = [(df_cis_1200.at[i, 'Longitude'], df_cis_1200.at[i, 'Latitude'],
                           1.2 * SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_1200
            pnt.style.iconstyle.scale = ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0

            # Counters
            i += 1

        # Save to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## -0.85 V

In [ ]:
k = 5  # Loop counter for type folders (5 = '-0.85 V')

# Goes through all '.csv' files
while j < len(filtered_data_list) and PLOT_3D:
    df_cis_850 = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[j]))
    last_index = df_cis_850.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_850.at[i, 'Station'] < feet_counter:
            pnt = type_folders[k].newpoint(name='-0.850', visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = 'Potential: -0.85 V'
            pnt.coords = [(df_cis_850.at[i, 'Longitude'], df_cis_850.at[i, 'Latitude'],
                           0.85 * SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_850
            pnt.style.iconstyle.scale = ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            pnt.style.linestyle.width = 0.01
            pnt.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, round(255 * 0.15))

            # Counters
            i += 1

        # Save to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## ACVG

In [ ]:
k = 6  # Loop counter for type folders (6 = "ACVG Indication (dB_V)")

# Goes through all '.csv' files
while j < len(filtered_data_list) and 'ACVG Indication (dB_V)' in df_cis.columns and PLOT_3D:
    df_cis_acvg = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[j]))
    df_cis_acvg = df_cis_acvg[df_cis_acvg['ACVG Indication (dB_V)'] != 0]
    df_cis_acvg = df_cis_acvg[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                               'On Potential', 'Off Potential',
                               'ACVG Indication (dB_V)', 'ACVG Notes']].reset_index(drop=True)
    last_index = df_cis_acvg.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_acvg.at[i, 'Station'] < feet_counter:
            # Break loop if there are no ACVG values
            if df_cis_acvg.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_acvg.at[i, 'ACVG Indication (dB_V)'],
                                           visibility=DATA_VISIBILITY)

            if pd.isnull(df_cis_acvg.at[i, 'ACVG Notes']):
                pnt.style.balloonstyle.text = (
                    f"ACVG Indication: {df_cis_acvg.at[i, 'ACVG Indication (dB_V)']} dB/V\n"
                    f"ID: {df_cis_acvg.index[i] + 1}\n"
                    f"Longitude: {df_cis_acvg.at[i, 'Longitude']}\n"
                    f"Latitude: {df_cis_acvg.at[i, 'Latitude']}\n"
                    f"Station (ft): {df_cis_acvg.at[i, 'Stationing (ft)']}")

            else:
                pnt.style.balloonstyle.text = (
                    f"ACVG Indication: {df_cis_acvg.at[i, 'ACVG Indication (dB_V)']} dB/V\n"
                    f"ID: {df_cis_acvg.index[i] + 1}\n"
                    f"Longitude: {df_cis_acvg.at[i, 'Longitude']}\n"
                    f"Latitude: {df_cis_acvg.at[i, 'Latitude']}\n"
                    f"Station (ft): {df_cis_acvg.at[i, 'Stationing (ft)']}\n"
                    f"Comment: {df_cis_acvg.at[i, 'ACVG Notes']}")

            pnt.coords = [(df_cis_acvg.at[i, 'Longitude'], df_cis_acvg.at[i, 'Latitude'],
                           df_cis_acvg.at[i, 'ACVG Indication (dB_V)'])]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_ACVG
            pnt.style.iconstyle.scale = ICON_SCALE + 0.5
            pnt.style.labelstyle.scale = 0

            # Counters
            i += 1

        # Save to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## PCM (Amps)

In [ ]:
k = 7  # Loop counter for type folders (7 = 'PCM Data (Amps)')

# Goes through all '.csv' files
while j < len(filtered_data_list) and 'PCM Data (Amps)' in df_cis.columns and PLOT_3D:
    df_cis_pcm_amps = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[j]))
    df_cis_pcm_amps = df_cis_pcm_amps[df_cis_pcm_amps['PCM Data (Amps)'] != 0]
    df_cis_pcm_amps = df_cis_pcm_amps[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                                       'PCM Data (Amps)']].reset_index(drop=True)
    last_index = df_cis_pcm_amps.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_pcm_amps.at[i, 'Station'] < feet_counter:
            # Break loop if there are no PCM (Amps) values
            if df_cis_pcm_amps.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_pcm_amps.at[i, 'PCM Data (Amps)'],
                                           visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"PCM: {df_cis_pcm_amps.at[i, 'PCM Data (Amps)']} Amps\n"
                f"Longitude: {df_cis_pcm_amps.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_pcm_amps.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_pcm_amps.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_pcm_amps.at[i, 'Longitude'], df_cis_pcm_amps.at[i, 'Latitude'],
                           df_cis_pcm_amps.at[i, 'PCM Data (Amps)'] * SCALE_PCM)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_PCM
            pnt.style.iconstyle.scale = ICON_SCALE + 0.5
            pnt.style.labelstyle.scale = 0

            # Counters
            i += 1

        # Save to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## PCM (%)

In [ ]:
k = 8  # Loop counter for type folders (8 = 'PCM % Change')

# Goes through all '.csv' files
while j < len(filtered_data_list) and 'PCM % Change' in df_cis.columns and PLOT_3D:
    df_cis_pcm_percent = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[j]))
    df_cis_pcm_percent = df_cis_pcm_percent[df_cis_pcm_percent['PCM % Change'] != 0]
    df_cis_pcm_percent = df_cis_pcm_percent[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                                             'PCM % Change']].reset_index(drop=True)
    last_index = df_cis_pcm_percent.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_pcm_percent.at[i, 'Station'] < feet_counter:
            # Break loop if there are no PCM (%) values
            if df_cis_pcm_percent.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_pcm_percent.at[i, 'PCM % Change'],
                                           visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"PCM: {df_cis_pcm_percent.at[i, 'PCM % Change']} %\n"
                f"Longitude: {df_cis_pcm_percent.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_pcm_percent.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_pcm_percent.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_pcm_percent.at[i, 'Longitude'],
                           df_cis_pcm_percent.at[i, 'Latitude'],
                           SCALE_PCM_PERCENT + df_cis_pcm_percent.at[i, 'PCM % Change'] * 0.5)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_PCM_PERCENT
            pnt.style.iconstyle.scale = ICON_SCALE + 0.5
            pnt.style.labelstyle.scale = 0

            # Counters
            i += 1

        # Save to '.kmz'
        kmz_name = f"{filtered_data_list[j].split('.csv', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(OUTPUT_DIR, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## Assign Station Numbers to KMZs

In [ ]:
# List of SNXXXX+XX ranges
sn_list = [''] * (math.ceil(total_miles_list[0]) * 2)

# Last index
last_index_sn = len(sn_list) - 1

# List of '.kmz' file names ((math.ceil(total_miles_list[0]) + 1???)
name_list = [''] * (math.ceil(total_miles_list[0]))

# List of stations as a nummber
station_list = np.arange(0, math.ceil(total_miles_list[0]) + 1)

# Last index
last_index_station = len(station_list) - 1

# First SN (number)
station_list[0] = starting_stationing

In [ ]:
# Assign a value of mile (ft) per element
for i in range(1, last_index_station + 1):
    station_list[i] = station_list[i - 1] + 5280

# Last SN (number)
station_list[last_index_station] = math.ceil(
    ((total_miles_list[0] - math.floor(total_miles_list[0])) * 5280 +
     station_list[last_index_station - 1]))

In [ ]:
# First SN
closest_index = (df_cis['Station'] + starting_stationing - station_list[0]).abs().argmin()
sn_list[0] = f"SN{df_cis['Stationing (ft)'].loc[closest_index]}".replace(' ', '')

In [ ]:
j = 1  # station_list counter
i = 1  # sn_list counter

# In between SN
while i < last_index_sn:
    closest_index = (df_cis['Station'] + starting_stationing - station_list[j]).abs().argmin()

    if df_cis['Station'].loc[closest_index] - station_list[j] >= 0:
        # Below closest index
        sn_list[i] = f"SN{df_cis['Stationing (ft)'].loc[closest_index - 1]}".replace(' ', '')
        sn_list[i + 1] = f"SN{df_cis['Stationing (ft)'].loc[closest_index]}".replace(' ', '')

    else:
        # Above closest index
        sn_list[i] = f"SN{df_cis['Stationing (ft)'].loc[closest_index]}".replace(' ', '')
        sn_list[i + 1] = f"SN{df_cis['Stationing (ft)'].loc[closest_index + 1]}".replace(' ', '')

    # Counters
    j += 1
    i += 2

In [ ]:
# Last SN
closest_index = (df_cis['Station'] + starting_stationing -
                 station_list[last_index_station]).abs().argmin()
sn_list[last_index_sn] = f"SN{df_cis['Stationing (ft)'].loc[closest_index]}".replace(' ', '')

In [ ]:
# Log
with open(LOG_DIR, 'a') as f:
    f.write("Removed files:")

# Remove files below certain byte size
for filename in os.listdir(OUTPUT_DIR):
    file_path = os.path.join(OUTPUT_DIR, filename)

    # Get file size in bytes
    size = os.path.getsize(file_path)

    # Remove files
    if size < 50 and filename != 'Log.txt':
        os.remove(file_path)
        with open(LOG_DIR, 'a') as f:
            f.write(f'\n- {filename}')

In [ ]:
# Remove duplicate SNs
sn_list = list(OrderedDict.fromkeys(sn_list))

# Get new index
last_index_sn = len(sn_list) - 1

In [ ]:
# Sort kmz files
kmz_files = glob.glob(os.path.join(OUTPUT_DIR, '*.kmz'))
kmz_files = [os.path.basename(file) for file in kmz_files]
kmz_files = natsorted(kmz_files)

In [ ]:
j = 0  # sn_list counter

# Name station number ranges
for i in range(0, last_index_station):
    name_list[i] = (f"{filtered_data_list[0].split('SN')[0][:-1]} "
                    f"({sn_list[j]} TO {sn_list[j + 1]}).kmz")

    # Counter
    j += 2

# Rename '.kmz' files
for i in range(0, last_index_station):
    os.rename(os.path.join(OUTPUT_DIR, kmz_files[i]), os.path.join(OUTPUT_DIR, name_list[i]))

# Exception Report

In [ ]:
# TODO: Add to exception report when Off is more negative than On
# TODO: Add to exception report where skips exist
# TODO: Add to exception report between TS

## Less Negative than -0.85V "On"

In [ ]:
# Combines all '.csv' files in folder
df_cis_filtered = pd.concat(
    (pd.read_csv(f) for f in glob.glob(os.path.join(DATA_DIR, r'*.csv'))), ignore_index=True)
df_cis_filtered = df_cis_filtered[df_cis_filtered['On Potential'] != 0]
df_cis_filtered = (df_cis_filtered[df_cis_filtered.columns.intersection(
    ['Station', 'Stationing (ft)', 'Longitude', 'Latitude', 'On Potential',
     'ACVG Indication (dB_V)'])].reset_index(drop=True))
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [ ]:
i = 1  # Loop counter for rows

if df_cis_filtered.shape[0] != 0:
    # Finds crossing points
    while i < last_index:
        # First data point
        if i == 1 and df_cis_filtered.at[0, 'On Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Second data point
        elif i == 2 and df_cis_filtered.at[1, 'On Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Last data point
        elif (i == last_index - 1 and
              df_cis_filtered.at[last_index, 'On Potential'] >= -0.85):
            df_cis_filtered.at[last_index, 'Crossing Point'] = 'X'

        # _'_
        elif ((df_cis_filtered.at[i - 1, 'On Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'On Potential'] <= -0.85) and
              (df_cis_filtered.at[i + 1, 'On Potential'] > -0.85) and
              i >= 1):
            df_cis_filtered.at[i, 'Crossing Point'] = 'XX'

        # '_
        elif ((df_cis_filtered.at[i + 1, 'On Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'On Potential'] <= -0.85)):
            df_cis_filtered.at[i, 'Crossing Point'] = 'X'

        # _'
        elif ((df_cis_filtered.at[i + 1, 'On Potential'] < -0.85) and
              (df_cis_filtered.at[i, 'On Potential'] >= -0.85)):
            df_cis_filtered.at[i + 1, 'Crossing Point'] = 'X'

        # Counters
        i += 1

    # Drops rows that are empty
    df_cis_filtered = (df_cis_filtered[df_cis_filtered['Crossing Point'] != '']
                       .reset_index(drop=True))

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
    .repeat(df_cis_filtered['Crossing Point'].isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index() + 1) / 2)

    # Create report dataframe
    df_cis_report = pd.DataFrame(
        index=np.arange(last_index),
        columns=['Station', 'Station Number', 'Latitude', 'Longitude',
                 'Station', 'Station Number', 'Latitude', 'Longitude'])
    df_cis_report['Length (ft)'] = 0.0

    if 'ACVG Indication (dB_V)' in df_cis.columns:
        df_cis_report['ACVG Max (dB/V)'] = 0.0

    else:
        df_cis_report['ACVG Max (dB/V)'] = ''

    df_cis_report['Comments'] = ''

    # Counters
    i = 0
    j = 0

    # Structure data
    while j < int((df_cis_filtered.last_valid_index() + 1) / 2):
        # Start
        df_cis_report.iat[j, 0] = df_cis_filtered.at[i, 'Station']
        df_cis_report.iat[j, 1] = df_cis_filtered.at[i, 'Stationing (ft)']
        df_cis_report.iat[j, 2] = df_cis_filtered.at[i, 'Latitude']
        df_cis_report.iat[j, 3] = df_cis_filtered.at[i, 'Longitude']

        # End
        df_cis_report.iat[j, 4] = df_cis_filtered.at[i + 1, 'Station']
        df_cis_report.iat[j, 5] = df_cis_filtered.at[i + 1, 'Stationing (ft)']
        df_cis_report.iat[j, 6] = df_cis_filtered.at[i + 1, 'Latitude']
        df_cis_report.iat[j, 7] = df_cis_filtered.at[i + 1, 'Longitude']

        # Length
        df_cis_report.iat[j, 8] = df_cis_report.iat[j, 4] - df_cis_report.iat[j, 0]

        # ACVG
        if 'ACVG Indication (dB_V)' in df_cis.columns:
            mask = ((df_cis['Station'] >= df_cis_filtered.at[i, 'Station']) &
                    (df_cis['Station'] <= df_cis_filtered.at[i + 1, 'Station']) &
                    (df_cis['ACVG Indication (dB_V)'] >= 45))

            df_cis_report.iat[j, 9] = df_cis.loc[mask, 'ACVG Indication (dB_V)'].max()

        # Counters
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)

    # Save to excel
    exception_excel = f"CIS Exception Report ({filtered_data_list[0].split('.csv')[0]}).xlsx"
    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, exception_excel),
                        mode='w', engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4, sheet_name='Less Negative than -0.85V (On)',
                               index=False)

In [ ]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(os.path.join(OUTPUT_DIR, exception_excel))
    sheet = wb['Less Negative than -0.85V (On)']

    # Company name with pipeline ID
    sheet.cell(1, 1).value = f"{CLIENT} ({filtered_data_list[0].split('SN')[0][:-1]})"

    # Station numbers surveyed
    sheet.cell(2, 1).value = (
        f"{filtered_data_list[0].split('SN')[1].split()[0]} to "
        f"{filtered_data_list[0].split('SN')[2].split('.')[0]}")

    # Start
    sheet.cell(4, 1).value = 'Start'

    # End
    sheet.cell(4, 4).value = 'End'

    # Total pipeline distance with issues
    total_feet = round(sum(total_miles_list * 5280))
    total_miles = round(sum(total_miles_list), 2)
    sheet.cell(last_index + 7, 1).value = (
        f"Total pipeline distance surveyed = {total_feet} feet or {total_miles} miles")
    total_length = round(df_cis_report['Length (ft)'].sum(), 2)
    length_percent = round(total_length / sum(total_miles_list * 5280) * 100, 2)
    sheet.cell(last_index + 8, 1).value = (
        f"Total pipeline distance less negative than -0.85V 'On' = "
        f"{total_length} feet ({length_percent} %)")

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(4, 9).font = Font(bold=True)
    sheet.cell(last_index + 7, 1).font = Font(bold=True)
    sheet.cell(last_index + 8, 1).font = Font(bold=True)

    # Merging
    sheet.merge_cells('A1:I1')
    sheet.merge_cells('A2:I2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')
    sheet.cell(4, 9).value = sheet.cell(5, 9).value
    sheet.merge_cells('I4:I5')

    # Alignment
    i = 0

    while i < (last_index + 5):
        for c in sheet['A1:I' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')

        # Counters
        i += 1

    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:I' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)

            # Counters
            i += 1

        for c in sheet['A1:I' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Column widths
    for col in 'ABCDEFGH':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['I'].width = 50

In [ ]:
# Save to excel
if df_cis_filtered.shape[0] != 0:
    wb.save(os.path.join(OUTPUT_DIR, exception_excel))

## Less Negative than -0.85V "Off"

In [ ]:
# Combines all '.csv' files at specified folder
df_cis_filtered = pd.concat(
    (pd.read_csv(f) for f in glob.glob(os.path.join(DATA_DIR, r'*.csv'))), ignore_index=True)
df_cis_filtered = df_cis_filtered[df_cis_filtered['Off Potential'] != 0]
df_cis_filtered = (df_cis_filtered[df_cis_filtered.columns.intersection(
    ['Station', 'Stationing (ft)', 'Longitude', 'Latitude', 'Off Potential',
     'ACVG Indication (dB_V)'])].reset_index(drop=True))
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [ ]:
i = 1  # Loop counter for rows

if df_cis_filtered.shape[0] != 0:
    # Finds crossing points
    while i < last_index:
        # First data point
        if i == 1 and df_cis_filtered.at[0, 'Off Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Second data point
        elif i == 2 and df_cis_filtered.at[1, 'Off Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Last data point
        elif (i == last_index - 1 and
              df_cis_filtered.at[last_index, 'Off Potential'] >= -0.85):
            df_cis_filtered.at[last_index, 'Crossing Point'] = 'X'

        # _'_
        elif ((df_cis_filtered.at[i - 1, 'Off Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'Off Potential'] <= -0.85) and
              (df_cis_filtered.at[i + 1, 'Off Potential'] > -0.85) and
              i >= 1):
            df_cis_filtered.at[i, 'Crossing Point'] = 'XX'

        # '_
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'Off Potential'] <= -0.85)):
            df_cis_filtered.at[i, 'Crossing Point'] = 'X'

        # _'
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] < -0.85) and
              (df_cis_filtered.at[i, 'Off Potential'] >= -0.85)):
            df_cis_filtered.at[i + 1, 'Crossing Point'] = 'X'

        # Counters
        i += 1

    df_cis_filtered = (df_cis_filtered[df_cis_filtered['Crossing Point'] != '']
                       .reset_index(drop=True))

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
    .repeat(df_cis_filtered['Crossing Point'].isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index() + 1) / 2)

    # Create report dataframe
    df_cis_report = pd.DataFrame(
        index=np.arange(last_index),
        columns=['Station', 'Station Number', 'Latitude', 'Longitude',
                 'Station', 'Station Number', 'Latitude', 'Longitude']
    )
    df_cis_report['Length (ft)'] = 0.0

    if 'ACVG Indication (dB_V)' in df_cis.columns:
        df_cis_report['ACVG Max (dB/V)'] = 0.0

    else:
        df_cis_report['ACVG Max (dB/V)'] = ''

    df_cis_report['Comments'] = ''

    # Counters
    i = 0
    j = 0

    # Structure data
    while j < int((df_cis_filtered.last_valid_index() + 1) / 2):
        # Start
        df_cis_report.iat[j, 0] = df_cis_filtered.at[i, 'Station']
        df_cis_report.iat[j, 1] = df_cis_filtered.at[i, 'Stationing (ft)']
        df_cis_report.iat[j, 2] = df_cis_filtered.at[i, 'Latitude']
        df_cis_report.iat[j, 3] = df_cis_filtered.at[i, 'Longitude']

        # End
        df_cis_report.iat[j, 4] = df_cis_filtered.at[i + 1, 'Station']
        df_cis_report.iat[j, 5] = df_cis_filtered.at[i + 1, 'Stationing (ft)']
        df_cis_report.iat[j, 6] = df_cis_filtered.at[i + 1, 'Latitude']
        df_cis_report.iat[j, 7] = df_cis_filtered.at[i + 1, 'Longitude']

        # Length
        df_cis_report.iat[j, 8] = df_cis_report.iat[j, 4] - df_cis_report.iat[j, 0]

        # ACVG
        if 'ACVG Indication (dB_V)' in df_cis.columns:
            mask = ((df_cis['Station'] >= df_cis_filtered.at[i, 'Station']) &
                    (df_cis['Station'] <= df_cis_filtered.at[i + 1, 'Station']) &
                    (df_cis['ACVG Indication (dB_V)'] >= 45))

            df_cis_report.iat[j, 9] = df_cis.loc[mask, 'ACVG Indication (dB_V)'].max()

        # Counters
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)

    # Save to file
    exception_excel = f"CIS Exception Report ({filtered_data_list[0].split('.csv')[0]}).xlsx"
    writing_mode = 'w'

    if os.path.exists(os.path.join(OUTPUT_DIR, exception_excel)):
        writing_mode = 'a'

    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, exception_excel),
                        mode=writing_mode, engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4, sheet_name='Less Negative than -0.85V (Off)',
                               index=False)

In [ ]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(os.path.join(OUTPUT_DIR, exception_excel))
    sheet = wb['Less Negative than -0.85V (Off)']

    # Company name with pipeline ID
    sheet.cell(1, 1).value = f"{CLIENT} ({filtered_data_list[0].split('SN')[0][:-1]})"

    # Station numbers surveyed
    sheet.cell(2, 1).value = (
        f"{filtered_data_list[0].split('SN')[1].split()[0]} to "
        f"{filtered_data_list[0].split('SN')[2].split('.')[0]}"
    )

    # Start
    sheet.cell(4, 1).value = 'Start'

    # End
    sheet.cell(4, 4).value = 'End'

    # Total pipeline distance with issues
    total_feet = round(sum(total_miles_list * 5280))
    total_miles = round(sum(total_miles_list), 2)
    sheet.cell(last_index + 7, 1).value = (
        f"Total pipeline distance surveyed = {total_feet} feet or {total_miles} miles"
    )
    total_length = round(df_cis_report['Length (ft)'].sum(), 2)
    length_percent = round(total_length / sum(total_miles_list * 5280) * 100, 2)
    sheet.cell(last_index + 8, 1).value = (
        f"Total pipeline distance less negative than -0.85V 'Off' = "
        f"{total_length} feet ({length_percent} %)"
    )

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(4, 9).font = Font(bold=True)
    sheet.cell(last_index + 7, 1).font = Font(bold=True)
    sheet.cell(last_index + 8, 1).font = Font(bold=True)

    # Merging
    sheet.merge_cells('A1:I1')
    sheet.merge_cells('A2:I2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')
    sheet.cell(4, 9).value = sheet.cell(5, 9).value
    sheet.merge_cells('I4:I5')

    # Alignment
    i = 0

    while i < (last_index + 5):
        for c in sheet['A1:I' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')

        # Counters
        i += 1

    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:I' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)

            # Counters
            i += 1

        for c in sheet['A1:I' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Column widths
    for col in 'ABCDEFGH':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['I'].width = 50

In [ ]:
# Save to excel
if df_cis_filtered.shape[0] != 0:
    wb.save(os.path.join(OUTPUT_DIR, exception_excel))

## More Negative than -1.2V "Off"

In [ ]:
# Combines all '.csv' files at specified folder
df_cis_filtered = pd.concat(
    (pd.read_csv(f) for f in glob.glob(os.path.join(DATA_DIR, r'*.csv'))),
    ignore_index=True
)
df_cis_filtered = df_cis_filtered[df_cis_filtered['Off Potential'] != 0]
df_cis_filtered = df_cis_filtered[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                                   'Off Potential']].reset_index(drop=True)
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [ ]:
i = 1  # Loop counter for rows

# Finds crossing points
if df_cis_filtered.shape[0] != 0:
    while i < last_index:
        # First data point
        if i == 1 and df_cis_filtered.at[0, 'Off Potential'] <= -1.2:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Second data point
        elif i == 2 and df_cis_filtered.at[1, 'Off Potential'] <= -1.2:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Last data point
        elif (i == last_index - 1 and
              df_cis_filtered.at[last_index, 'Off Potential'] <= -1.2):
            df_cis_filtered.at[last_index, 'Crossing Point'] = 'X'

        # '_'
        elif ((df_cis_filtered.at[i - 1, 'Off Potential'] < -1.2) and
              (df_cis_filtered.at[i, 'Off Potential'] >= -1.2) and
              (df_cis_filtered.at[i + 1, 'Off Potential'] < -1.2) and
              i >= 1):
            df_cis_filtered.at[i, 'Crossing Point'] = 'XX'

        # '_
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] > -1.2) and
              (df_cis_filtered.at[i, 'Off Potential'] <= -1.2)):
            df_cis_filtered.at[i + 1, 'Crossing Point'] = 'X'

        # _'
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] < -1.2) and
              (df_cis_filtered.at[i, 'Off Potential'] >= -1.2)):
            df_cis_filtered.at[i, 'Crossing Point'] = 'X'

        # Counters
        i += 1

    df_cis_filtered = (df_cis_filtered[df_cis_filtered['Crossing Point'] != '']
                       .reset_index(drop=True))

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
    .repeat(df_cis_filtered['Crossing Point'].isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index() + 1) / 2)

    # Create report dataframe
    df_cis_report = pd.DataFrame(
        index=np.arange(last_index),
        columns=['Station', 'Station Number', 'Latitude', 'Longitude',
                 'Station', 'Station Number', 'Latitude', 'Longitude']
    )
    df_cis_report['Length (ft)'] = 0.0
    df_cis_report['Comments'] = ''

    # Counters
    i = 0
    j = 0

    # Structure data
    while j < int((df_cis_filtered.last_valid_index() + 1) / 2):
        # Start
        df_cis_report.iat[j, 0] = df_cis_filtered.at[i, 'Station']
        df_cis_report.iat[j, 1] = df_cis_filtered.at[i, 'Stationing (ft)']
        df_cis_report.iat[j, 2] = df_cis_filtered.at[i, 'Latitude']
        df_cis_report.iat[j, 3] = df_cis_filtered.at[i, 'Longitude']

        # End
        df_cis_report.iat[j, 4] = df_cis_filtered.at[i + 1, 'Station']
        df_cis_report.iat[j, 5] = df_cis_filtered.at[i + 1, 'Stationing (ft)']
        df_cis_report.iat[j, 6] = df_cis_filtered.at[i + 1, 'Latitude']
        df_cis_report.iat[j, 7] = df_cis_filtered.at[i + 1, 'Longitude']

        # Length
        df_cis_report.iat[j, 8] = df_cis_report.iat[j, 4] - df_cis_report.iat[j, 0]

        # Counters
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)

    # Save to file
    exception_excel = f"CIS Exception Report ({filtered_data_list[0].split('.csv')[0]}).xlsx"
    writing_mode = 'w'

    if os.path.exists(os.path.join(OUTPUT_DIR, exception_excel)):
        writing_mode = 'a'

    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, exception_excel),
                        mode=writing_mode, engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4, sheet_name='More Negative than -1.2V (Off)',
                               index=False)

In [ ]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(os.path.join(OUTPUT_DIR, exception_excel))
    sheet = wb['More Negative than -1.2V (Off)']

    # Company name with pipeline ID
    sheet.cell(1, 1).value = f"{CLIENT} ({filtered_data_list[0].split('SN')[0][:-1]})"

    # Station numbers surveyed
    sheet.cell(2, 1).value = (
        f"{filtered_data_list[0].split('SN')[1].split()[0]} to "
        f"{filtered_data_list[0].split('SN')[2].split('.')[0]}"
    )

    # Start
    sheet.cell(4, 1).value = 'Start'

    # End
    sheet.cell(4, 4).value = 'End'

    # Total pipeline distance with issues
    total_feet = round(sum(total_miles_list * 5280))
    total_miles = round(sum(total_miles_list), 2)
    sheet.cell(last_index + 7, 1).value = (
        f"Total pipeline distance surveyed = {total_feet} feet or {total_miles} miles"
    )
    total_length = round(df_cis_report['Length (ft)'].sum(), 2)
    length_percent = round(total_length / sum(total_miles_list * 5280) * 100, 2)
    sheet.cell(last_index + 8, 1).value = (
        f"Total pipeline distance more negative than -1.2V 'Off' = "
        f"{total_length} feet ({length_percent} %)"
    )

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(last_index + 7, 1).font = Font(bold=True)
    sheet.cell(last_index + 8, 1).font = Font(bold=True)

    # Merging
    sheet.merge_cells('A1:H1')
    sheet.merge_cells('A2:H2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')

    # Alignment
    i = 0

    while i < (last_index + 5):
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')

        # Counters
        i += 1

    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:H' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)

            # Counters
            i += 1

        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

In [ ]:
# Save to excel
if df_cis_filtered.shape[0] != 0:
    wb.save(os.path.join(OUTPUT_DIR, exception_excel))

# ACVG Report

In [ ]:
if not PLOT_3D:
    # Dataframe
    df_cis_acvg = pd.read_csv(os.path.join(DATA_DIR, filtered_data_list[0]))
    df_cis_acvg = df_cis_acvg[df_cis_acvg['ACVG Indication (dB_V)'] != 0]
    df_cis_acvg = df_cis_acvg[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                               'On Potential', 'Off Potential',
                               'ACVG Indication (dB_V)', 'ACVG Notes']].reset_index(drop=True)

In [ ]:
# Borders
thin = Side(border_style='thin', color='000000')
white_border = Side(border_style='thin', color='FFFFFF')

## -0.85V "On" (ACVG >= 45)

In [ ]:
if 'ACVG Indication (dB_V)' in df_cis.columns:
    # Dataframe
    df_cis_acvg_on_45 = df_cis_acvg.copy()
    df_cis_acvg_on_45 = df_cis_acvg_on_45[(df_cis_acvg_on_45['On Potential'] >= -0.85) &
                                          (df_cis_acvg_on_45['ACVG Indication (dB_V)'] >= 45.0)]
    df_cis_acvg_on_45 = df_cis_acvg_on_45.drop(columns='Station')
    last_index = df_cis_acvg_on_45.reset_index().last_valid_index()

    # Save to excel
    acvg_excel = f"ACVG Report ({filtered_data_list[0].split('.csv')[0]}).xlsx"
    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, acvg_excel), mode='w',
                        engine='openpyxl') as writer:
        df_cis_acvg_on_45.to_excel(writer, sheet_name='-0.85V (On)(ACVG 45)', index_label='ID')

    # Modify sheet
    wb = load_workbook(os.path.join(OUTPUT_DIR, acvg_excel))
    sheet = wb['-0.85V (On)(ACVG 45)']

    # Rename ACVG column
    sheet.cell(1, 7).value = 'ACVG (dB/V)'

    # Formatting
    i = 0

    while i < last_index + 2:
        for c in sheet['A1:H' + str(last_index + 2)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
            c.font = Font(bold=False)
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Header
    for i in range(1, 9):
        sheet.cell(1, i).font = Font(bold=True)

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

    # Save to excel
    wb.save(os.path.join(OUTPUT_DIR, acvg_excel))

## -0.85V "On" (ACVG >= 60)

In [ ]:
if 'ACVG Indication (dB_V)' in df_cis.columns:
    # Dataframe
    df_cis_acvg_on_60 = df_cis_acvg.copy()
    df_cis_acvg_on_60 = df_cis_acvg_on_60[(df_cis_acvg_on_60['On Potential'] >= -0.85) &
                                          (df_cis_acvg_on_60['ACVG Indication (dB_V)'] >= 60.0)]
    df_cis_acvg_on_60 = df_cis_acvg_on_60.drop(columns='Station')
    last_index = df_cis_acvg_on_60.reset_index().last_valid_index()

    # Save to excel
    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, acvg_excel), mode='a',
                        engine='openpyxl') as writer:
        df_cis_acvg_on_60.to_excel(writer, sheet_name='-0.85V (On)(ACVG 60)', index_label='ID')

    # Modify sheet
    wb = load_workbook(os.path.join(OUTPUT_DIR, acvg_excel))
    sheet = wb['-0.85V (On)(ACVG 60)']

    # Rename ACVG column
    sheet.cell(1, 7).value = 'ACVG (dB/V)'

    # Formatting
    i = 0

    while i < last_index + 2:
        for c in sheet['A1:H' + str(last_index + 2)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
            c.font = Font(bold=False)
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Header
    for i in range(1, 9):
        sheet.cell(1, i).font = Font(bold=True)

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

    # Save to excel
    wb.save(os.path.join(OUTPUT_DIR, acvg_excel))

## -0.85V "Off" (ACVG >= 45)

In [ ]:
if 'ACVG Indication (dB_V)' in df_cis.columns:
    # Dataframe
    df_cis_acvg_off_45 = df_cis_acvg.copy()
    df_cis_acvg_off_45 = df_cis_acvg_off_45[(df_cis_acvg_off_45['Off Potential'] >= -0.85) &
                                            (df_cis_acvg_off_45['ACVG Indication (dB_V)'] >= 45.0)]
    df_cis_acvg_off_45 = df_cis_acvg_off_45.drop(columns='Station')
    last_index = df_cis_acvg_off_45.reset_index().last_valid_index()

    # Save to excel
    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, acvg_excel), mode='a',
                        engine='openpyxl') as writer:
        df_cis_acvg_off_45.to_excel(writer, sheet_name='-0.85V (Off)(ACVG 45)', index_label='ID')

    # Modify sheet
    wb = load_workbook(os.path.join(OUTPUT_DIR, acvg_excel))
    sheet = wb['-0.85V (Off)(ACVG 45)']

    # Rename ACVG column
    sheet.cell(1, 7).value = 'ACVG (dB/V)'

    # Formatting
    i = 0

    while i < last_index + 2:
        for c in sheet['A1:H' + str(last_index + 2)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
            c.font = Font(bold=False)
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Header
    for i in range(1, 9):
        sheet.cell(1, i).font = Font(bold=True)

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

    # Save to excel
    wb.save(os.path.join(OUTPUT_DIR, acvg_excel))

## -0.85V "Off" (ACVG >= 60)

In [ ]:
if 'ACVG Indication (dB_V)' in df_cis.columns:
    # Dataframe
    df_cis_acvg_off_60 = df_cis_acvg.copy()
    df_cis_acvg_off_60 = df_cis_acvg_off_60[(df_cis_acvg_off_60['Off Potential'] >= -0.85) &
                                            (df_cis_acvg_off_60['ACVG Indication (dB_V)'] >= 60.0)]
    df_cis_acvg_off_60 = df_cis_acvg_off_60.drop(columns='Station')
    last_index = df_cis_acvg_off_60.reset_index().last_valid_index()

    # Save to excel
    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, acvg_excel), mode='a',
                        engine='openpyxl') as writer:
        df_cis_acvg_off_60.to_excel(writer, sheet_name='-0.85V (Off)(ACVG 60)', index_label='ID')

    # Modify sheet
    wb = load_workbook(os.path.join(OUTPUT_DIR, acvg_excel))
    sheet = wb['-0.85V (Off)(ACVG 60)']

    # Rename ACVG column
    sheet.cell(1, 7).value = 'ACVG (dB/V)'

    # Formatting
    i = 0

    while i < last_index + 2:
        for c in sheet['A1:H' + str(last_index + 2)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
            c.font = Font(bold=False)
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Header
    for i in range(1, 9):
        sheet.cell(1, i).font = Font(bold=True)

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

    # Save to excel
    wb.save(os.path.join(OUTPUT_DIR, acvg_excel))

# Severity Matrix

In [ ]:
# TODO: Deal with this when there is no acvg, etc.

In [ ]:
# # Dataframe
# df_cis_matrix = df_cis.copy()
# df_cis_matrix = df_cis_matrix[['Station', 'Stationing (ft)', 'On Potential', 'Off Potential',
#                                'Comments', 'Longitude', 'Latitude',
#                                'ACVG Indication (dB_V)', 'ACVG Notes',
#                                'PCM Data (Amps)', 'PCM % Change']].reset_index(drop=True)
# df_cis_matrix['ACVG Severity'] = 0
# df_cis_matrix['PCM Severity'] = 0
# df_cis_matrix['Total Severity'] = 0

In [ ]:
# # ACVG
# df_cis_matrix.loc[(df_cis_matrix['ACVG Indication (dB_V)'] < 50), 'ACVG Severity'] = 0
# df_cis_matrix.loc[(df_cis_matrix['ACVG Indication (dB_V)'] >= 50) & (
#         df_cis_matrix['ACVG Indication (dB_V)'] < 66), 'ACVG Severity'] = 1
# df_cis_matrix.loc[(df_cis_matrix['ACVG Indication (dB_V)'] >= 66) & (
#         df_cis_matrix['ACVG Indication (dB_V)'] < 81), 'ACVG Severity'] = 4
# df_cis_matrix.loc[(df_cis_matrix['ACVG Indication (dB_V)'] >= 81), 'ACVG Severity'] = 6
# # df_cis_matrix = df_cis_matrix[df_cis_matrix['ACVG Indication (dB_V)'] != 0]

In [ ]:
# # PCM
# df_cis_matrix.loc[(df_cis_matrix['PCM % Change'] >= 0) & (
#         df_cis_matrix['PCM % Change'] < 10), 'PCM Severity'] = 1
# df_cis_matrix.loc[(df_cis_matrix['PCM % Change'] >= 10) & (
#         df_cis_matrix['PCM % Change'] < 20), 'PCM Severity'] = 2
# df_cis_matrix.loc[(df_cis_matrix['PCM % Change'] >= 20), 'PCM Severity'] = 3
# # df_cis_matrix = df_cis_matrix[df_cis_matrix['PCM % Change'] != 0]

# Log File

In [ ]:
# Current date and time
e = datetime.datetime.now()
execution_time = time.time() - start_time
minutes, seconds = divmod(execution_time, 60)

# Modify log file with information of interest
with open(LOG_DIR, 'a') as f:
    f.write(f"\n\nGPS Accuracy {LOWER_CUTOFF}ft to {UPPER_CUTOFF}ft:\n")
    f.write(f"   - Count: {total_count}\n")
    f.write(f"   - Cutoff Count: {total_count - cutoff_count}\n")
    f.write(f"   - Consistency: {round(cutoff_count / total_count * 100, 1)}%\n")
    f.write(f"   - Mean: {round(mean, 3)}\n")
    f.write(f"   - Mode: {mode[0]}\n")
    f.write(f"   - Standard Deviation: {round(std, 3)}\n")

    if mean - std * 2.576 <= 0:
        f.write(f"   - 99% Confidence Interval: {0} to {round(mean + std * 2.576, 3)}\n")

    else:
        f.write(f"   - 99% Confidence Interval: {round(mean - std * 2.576, 3)} to "
                f"{round(mean + std * 2.576, 3)}\n")

    f.write(f"   - Outliers: {df_outliers['Distance (ft)'].count()}\n")

    if PLOT_3D:
        f.write(f"   - Duplicated GPS Pairs: {df_duplicates['Station'].value_counts().sum()}\n\n")

    if PLOT_3D:
        f.write(f"Measurements:\n")
        f.write(f"   - On: {on_measurements}\n")
        f.write(f"   - Off: {off_measurements}\n")
        f.write(f"   - Ratio (On/Off): {round(on_measurements / off_measurements, 3)}\n\n")

    f.write(f"User: {socket.gethostname()}\n")
    f.write(f'{e.strftime("%b, %d, %Y")}, {e.hour:02d}:{e.minute:02d}:{e.second:02d}')
    f.write('\n')
    f.write(f"Execution Time ({int(minutes)}:{int(seconds)})")

In [ ]:
# Log
os.rename(LOG_DIR, os.path.join(OUTPUT_DIR, 'Log (' +
                                filtered_data_list[0].split('.csv')[0] + ').txt'))

if PLOT_3D and df_duplicates.shape[0] > 0:
    # Duplicated GPS
    os.rename(DUPLICATES_DIR, os.path.join(OUTPUT_DIR, 'Duplicate GPS Pairs (' +
                                           filtered_data_list[0].split('.csv')[0] + ').csv'))